# 유튜브에서 크롤링한 댓글 전처리 진행하기

## Import library

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Okt

## 데이터 전처리 시작

### 데이터 불러오기
usecols을 이용해서 필요한 comment 부분만 가져왔다

In [50]:
data = pd.read_excel('1_말안듣는말.xlsx', usecols = ['comment'])
data

,comment
0,로봇말로 바꾸던지;; 말을 바꿔서 시도를 여러번 할수있게 해주던지;; 오늘 처음만난...
1,같은 말로 해야 형편성에 맞지 않냐? 이건 좀 아닌듯
2,말 죽이고 싶었겠다
3,말이 경기를 뛰는데 사람이 평가받는구만
4,당근 챙겨가면 안되나??
...,...
7119,@현호 ㄹㅇ 다시보니 ㅁ친놈임ㅋㅋ
7120,ㅋㅋㅋㅋ
7121,김유신 이야기는 ㅈㄴ 웃긴게 자기가 인사불성인상태에서 주인이 자주가는곳 잘 데리다줫...
7122,헐..


### 데이터 중복확인
데이터 중복있는지 확인하기 위해서 고유값만 출력하는 nunique() 사용

In [51]:
data['comment'].nunique()

6966

### 중복 제거

In [52]:
data.drop_duplicates(subset = ['comment'], inplace=True) # 중복 제거
data = data.dropna(how = 'any') # Null 값이 존재하는 행 제거

In [53]:
len(data) # 잘 제거되었는지 확인

6966

### 정규 표현식을 이용하여 한글, 공백을 제외하고 제거하기

In [54]:
data['comment'] = data['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

# 잘 되었구나
data[:10]

<ipython-input-54-aedb2aa454cb>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['comment'] = data['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,comment
0,로봇말로 바꾸던지 말을 바꿔서 시도를 여러번 할수있게 해주던지 오늘 처음만난 말새끠...
1,같은 말로 해야 형편성에 맞지 않냐 이건 좀 아닌듯
2,말 죽이고 싶었겠다
3,말이 경기를 뛰는데 사람이 평가받는구만
4,당근 챙겨가면 안되나
5,전라도 말인가 보네
6,말이 하기 싫다는데 하는것도 말이 안되죠
7,방구석 여포들 적토마 타는 소리하네
8,랜덤으로 하면 비리도 있을 것 같은 느낌 정말 단점들 투성이인데도 랜덤으로 하네요
9,운이 경기 성적에 많이 반영되면 로또도 올림픽에 넣지 그러냐


### 빈값 다시 확인
정규표현식을 이용해서 제거하게되면, 빈 값이 생길게 뻔히 보여서 빈값 확인 및 null값으로 만들어서 제거합니당

In [55]:
data['comment'].replace('', np.nan, inplace=True) # null값으로 만들어주기
print(data.isnull().sum()) # 총 몇 개인지 확인

comment    5
dtype: int64


In [56]:
# null 값을 가진 샘플 확인
data.loc[data.comment.isnull()]

,comment
255,NaN
303,NaN
3591,NaN
5151,NaN
5219,NaN


In [57]:
# null값아 잘가라~
data = data.dropna(how = 'any')

### 불용어 제거 및 토큰화

In [58]:
# 불용어 단계에서 "신조어"를 위주로 추출하려면 불용어 리스트를 더 추가하는 것이 좋을 듯 싶다 
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [60]:
# 불용어제거하면서 토큰화 okt 이용
okt = Okt()

res = []
for sentence in data['comment']:
    lis = []
    lis = okt.morphs(sentence, stem=True) # 토큰화
    lis = [word for word in lis if not word in stopwords] # 불용어 제거
    res.append(lis)

In [61]:
res

[['로봇',
  '말로',
  '바꾸다',
  '말',
  '을',
  '바꾸다',
  '시도',
  '여러',
  '번',
  '해주다',
  '오늘',
  '처음',
  '만',
  '난',
  '말새끠',
  '말',
  '을',
  '안',
  '듣다',
  '어쩌라고',
  'ㅠㅠ',
  '너무',
  '억울하다',
  '안타깝다',
  'ㅠㅠ'],
 ['같다', '말로', '형편', '성', '맞다', '않다', '이건', '아니다'],
 ['말', '죽', '이고', '싶다'],
 ['말', '경기', '뛰다', '사람', '평가', '받다'],
 ['당근', '챙기다', '안되다'],
 ['전라도', '말', '인가', '보다'],
 ['말', '싫다', '말', '안되다'],
 ['방', '구석', '여포', '적토마', '타다', '소리'],
 ['랜덤', '비리', '있다', '것', '같다', '느낌', '정말', '단점', '투성이', '인데', '랜덤'],
 ['운', '경기', '성적', '많이', '반영', '되다', '로또', '올림픽', '넣다', '그렇다'],
 ['말', '말', '을', '끼다', '놓다', '순위', '뒤죽박죽', '되다'],
 ['나다', '경기', '끝나다', '저', '말', '목', '찌르다', '죽이다'],
 ['근대',
  '종목',
  '비',
  '인기',
  '인',
  '이유',
  '알다',
  '스포츠',
  '무작위',
  '배정',
  '말',
  '이라는',
  '운빨',
  '요소',
  '들어가다',
  '말',
  '선택',
  '해주다',
  '사람',
  '만',
  '메달',
  '권',
  '가능하다'],
 ['적어도', '하루', '정도', '충분하다', '시간', '을', '주다', '어떻다'],
 ['말', '대신', '오토바이', '로', '바꾸다'],
 ['말타', '기다', '빼다', '차라리', '카레이싱', '을', '넣다'],
 ['말', 'ㅈㄴ',

### 토크나이저 사용
토크나이저를 사용해서 빈도를 추출한다

In [62]:
# 토크나이저 사용해서 빈도순으로 단어집합 ㄱㄱ
# fit_on_texts는 입력한 텍스트로부터 단어 빈도수가 높은 순으로 낮은 정수 인덱스를 부여함

tokenizer = Tokenizer()
tokenizer.fit_on_texts(res)

In [63]:
# 당연하겠지만 말이 젤 많이 나왔네요
print(tokenizer.word_index)

{'말': 1, '을': 2, '선수': 3, '아니다': 4, '있다': 5, '저': 6, '같다': 7, '종목': 8, '되다': 9, '보다': 10, '경기': 11, '승마': 12, '올림픽': 13, '안': 14, '사람': 15, '로': 16, '너무': 17, '에서': 18, '없다': 19, '다': 20, '교감': 21, '안타깝다': 22, '진짜': 23, '게': 24, '동물': 25, '않다': 26, '것': 27, '고': 28, '이다': 29, '그렇다': 30, '거': 31, '못': 32, '운': 33, '스포츠': 34, 'ㅋㅋ': 35, '안되다': 36, '그냥': 37, '좋다': 38, '실력': 39, '왜': 40, '운빨': 41, '받다': 42, '네': 43, '맞다': 44, '뭐': 45, '타다': 46, '이네': 47, '인데': 48, '듣다': 49, '들다': 50, '말고기': 51, '때리다': 52, '내': 53, '불쌍하다': 54, '자다': 55, '만': 56, '생각': 57, '나': 58, 'ㅠㅠ': 59, '수': 60, '적': 61, '전쟁': 62, '하고': 63, '무슨': 64, '아': 65, '그': 66, '더': 67, '인': 68, '주다': 69, '바꾸다': 70, '인간': 71, '저런': 72, '이건': 73, '다른': 74, '알다': 75, '시간': 76, '자기': 77, '먹다': 78, '얼마나': 79, '요': 80, '이렇다': 81, '싶다': 82, 'ㅋㅋㅋ': 83, 'ㅋㅋㅋㅋ': 84, '빼다': 85, '울다': 86, '문제': 87, '노력': 88, '나오다': 89, '룰': 90, '속상하다': 91, '라': 92, '버리다': 93, '임': 94, '가다': 95, '장애물': 96, '오다': 97, '시키다': 98, '한테': 99, '모르다': 100, '만들다': 101,

In [64]:
# 말이 몇번 나왔는지 한번 보겠습니다
print(tokenizer.word_counts['말'])

5112


In [67]:
# 신조어로 보이는 운빨망겜 얼마나 나왔는지 한번 보겠습니당
print(tokenizer.word_counts['운빨망겜'])

5


일단은 신조어를 뽑는다는 생각 말고 단순히 불용어 제거, 토큰화하고 빈도 추출하는 부분만 고려했습니다.

확인해 보니까 불용어에 기본적인 단어들이 너무 상위빈도를 차지해서, 기본 단어 모두 제거한 후에 빈도수를 추출해야 신조어를 추출할 수 있지 않을까 싶기도 합니다. ㅜ
